# スタサプRPA

In [1]:
# selenium 4
import math
import time
import os
import gc
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.wait import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

os.environ['WDM_SSL_VERIFY'] = '0'

# import selenium.webdriver.common.devtools.v114 as devtools
from selenium.webdriver.common.by import By

# .env file load
from dotenv import load_dotenv

load_dotenv()

True

# クラス作成

In [6]:
class StudysapuriSkip:
    def __init__(self, continue_login=True) -> None:
        self.options = webdriver.FirefoxOptions()
        # options.add_argument('--headless')

        self.driver = webdriver.Firefox(
            service=FirefoxService(GeckoDriverManager().install()), options=self.options
        )

        self.driver.set_window_position(0, 0)
        self.original_window = self.driver.current_window_handle
        self.load_credential()

        self._studysapuri_uri_dict = {
            "homework_active": "https://learn.studysapuri.jp/ja/todos/active",
            "homework_expired": "https://learn.studysapuri.jp/ja/todos/expired",
        }

        self.load_timeout_config()
        if continue_login:
            try:
                self.login()
                self.count_task()
            except TimeoutException as err:
                print(err)
                self.driver.get("data:text,Login failed.")
            else:
                print("Login Successful")

        self.__tab = []


    def load_timeout_config(self) -> None:
        self.timeout = os.getenv("TIMEOUT_SEC")
        if self.timeout == None:
            self.timeout = 20
        else:
            self.timeout = float(self.timeout)

    def load_credential(self) -> None:
        self.service_url = os.getenv("LOGIN_URL")
        self.__email = os.getenv("EMAIL_ADDRESS")
        self.__password = os.getenv("PASSWORD")

        # 環境変数未設定時
        if self.__email == None:
            print("Your ID is None.")
            self.__email = input("Please input ID or email:")

        if self.__password == None:
            print("Your PASSWORD is None.")
            self.__password = input("Please input password:")

    def release_credential(self):
        self.__email = self.__password = None

    def login(self):
        self.driver.get(self.service_url)
        self.username_input_element = self.driver.find_element(
            by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[2]/input'
        )
        self.password_input_element = self.driver.find_element(
            by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/div[3]/span/input'
        )
        self.login_button = self.driver.find_element(
            by=By.XPATH, value='//*[@id="root"]/div/div/div/form/div/button[2]/span[2]'
        )
        # ----------------------------------
        self.username_input_element.clear()
        self.password_input_element.clear()
        self.username_input_element.send_keys(self.__email)
        self.password_input_element.send_keys(self.__password)
        time.sleep(0.5)
        self.login_button.click()

    def count_task(self):
        self.driver.get(self._studysapuri_uri_dict["homework_active"])
        WebDriverWait(self.driver, timeout=self.timeout).until(
            lambda d: d.find_elements(
                by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
        )
        try:
            self.is_active_homework_empty = WebDriverWait(
                self.driver, timeout=4
            ).until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value="div > ul li > button"))
        except TimeoutException as err:
            self.is_active_homework_empty = []

        self.driver.get(self._studysapuri_uri_dict["homework_expired"])
        WebDriverWait(self.driver, timeout=self.timeout).until(
            lambda d: d.find_elements(
                by=By.CSS_SELECTOR, value="[class*=BasicTemplate__Body]")
        )
        try:
            self.is_expired_homework_empty = WebDriverWait(
                self.driver, timeout=4
            ).until(lambda d: d.find_elements(by=By.CSS_SELECTOR, value="div > ul li > button"))
        except TimeoutException as err:
            self.is_expired_homework_empty = []

        print(
            f"配信中の課題{len(self.is_active_homework_empty)}",
            f"期限切れ{len(self.is_expired_homework_empty)}"
        )

        return {
            "active_homework": len(self.is_active_homework_empty),
            "expired_homework": len(self.is_expired_homework_empty)
        }

    def first_taskwork_open(self):
        WebDriverWait(self.driver, timeout=self.timeout*2).until(
            lambda d: d.find_element(
                By.CSS_SELECTOR, 'button[class*="TodoCard"]'
            )
        )
        self.first_taskwork = self.driver.find_element(
            By.CSS_SELECTOR, value='button[class*="TodoCard"]'
        )
        self.first_taskwork.click()

        WebDriverWait(self.driver, timeout=self.timeout*5).until(
            expected_conditions.presence_of_all_elements_located(
                (By.CSS_SELECTOR, 'span[class*=isIncomplete]')
            )
        )

        self.first_todo = self.driver.find_element(
            By.CSS_SELECTOR, value='span[class*=isIncomplete]'
        )
        f = open("JS/incompleteClick.js", "r")
        try:
            self.driver.execute_script(f.read())
        except:
            print("An error has occurred.")
        finally:
            f.close()
            print("file resource released.")

    def process_todo(self):
        self.driver

        

In [3]:
instans = StudysapuriSkip()

c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
[WDM] - Downloading: 17.8kB [00:00, 18.8MB/s]                   
c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Python311\Lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Python311\Lib\site-packages\urlli

配信中の課題0 期限切れ20
Login Successful


In [5]:
print("30s 後に終了します。")
time.sleep(30)
instans.driver.quit()